### Vectorization of functions

* functions are typically not "vectorized" in Julia
* you have to explicitly use the "dot-notation"

In [1]:
x = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [3]:
sin.(x) == broadcast(sin,x)

3-element Array{Float64,1}:
 0.8414709848078965
 0.9092974268256817
 0.1411200080598672

In [4]:
?broadcast

search: broadcast broadcast! Broadcast



```
broadcast(f, As...)
```

Broadcast the function `f` over the arrays, tuples, collections, [`Ref`](@ref)s and/or scalars `As`.

Broadcasting applies the function `f` over the elements of the container arguments and the scalars themselves in `As`. Singleton and missing dimensions are expanded to match the extents of the other arguments by virtually repeating the value. By default, only a limited number of types are considered scalars, including `Number`s, `String`s, `Symbol`s, `Type`s, `Function`s and some common singletons like [`missing`](@ref) and [`nothing`](@ref). All other arguments are iterated over or indexed into elementwise.

The resulting container type is established by the following rules:

  * If all the arguments are scalars or zero-dimensional arrays, it returns an unwrapped scalar.
  * If at least one argument is a tuple and all others are scalars or zero-dimensional arrays, it returns a tuple.
  * All other combinations of arguments default to returning an `Array`, but custom container types can define their own implementation and promotion-like rules to customize the result when they appear as arguments.

A special syntax exists for broadcasting: `f.(args...)` is equivalent to `broadcast(f, args...)`, and nested `f.(g.(args...))` calls are fused into a single broadcast loop.

# Examples

```jldoctest
julia> A = [1, 2, 3, 4, 5]
5-element Array{Int64,1}:
 1
 2
 3
 4
 5

julia> B = [1 2; 3 4; 5 6; 7 8; 9 10]
5×2 Array{Int64,2}:
 1   2
 3   4
 5   6
 7   8
 9  10

julia> broadcast(+, A, B)
5×2 Array{Int64,2}:
  2   3
  5   6
  8   9
 11  12
 14  15

julia> parse.(Int, ["1", "2"])
2-element Array{Int64,1}:
 1
 2

julia> abs.((1, -2))
(1, 2)

julia> broadcast(+, 1.0, (0, -2.0))
(1.0, -1.0)

julia> (+).([[0,2], [1,3]], Ref{Vector{Int}}([1,-1]))
2-element Array{Array{Int64,1},1}:
 [1, 1]
 [2, 2]

julia> string.(("one","two","three","four"), ": ", 1:4)
4-element Array{String,1}:
 "one: 1"
 "two: 2"
 "three: 3"
 "four: 4"

```


### Broadcasting different shapes

* dimensions of length 1 will be extended
* can be used e.g. to subtract columns means
* or for outer products
* left-hand side arrays can be written to using .=

In [7]:
x = ones(10,1)
y = rand(10,5)
x .+ y

10×5 Array{Float64,2}:
 1.81507  1.50386  1.98598  1.33814  1.37277
 1.21444  1.50733  1.66818  1.02294  1.14256
 1.42187  1.21092  1.89536  1.48548  1.1476 
 1.68698  1.02048  1.33966  1.14762  1.74976
 1.01351  1.57467  1.29245  1.4364   1.68125
 1.74868  1.15224  1.19412  1.24843  1.49699
 1.62877  1.5365   1.34366  1.12102  1.07493
 1.17499  1.37084  1.98403  1.51868  1.39306
 1.52285  1.38033  1.307    1.5444   1.12841
 1.62548  1.8075   1.37483  1.96554  1.85816

In [12]:
using Statistics
y
(y .- mean(y,dims=1))./std(y,dims=1)

10×5 Array{Float64,2}:
  1.22095    0.421947   1.43103   -0.16381   -0.111511 
 -1.00258    0.436994   0.414642  -1.31819   -0.919428 
 -0.234692  -0.847182   1.14122    0.375816  -0.901741 
  0.746755  -1.67225   -0.636007  -0.861583   1.2115   
 -1.74644    0.728725  -0.786991   0.196071   0.971067 
  0.975177  -1.10142   -1.10148   -0.492369   0.324404 
  0.531247   0.563345  -0.623208  -0.95898   -1.15679  
 -1.14863   -0.154348   1.42478    0.497405  -0.0403067
  0.139142  -0.113238  -0.740446   0.591632  -0.96911  
  0.519073   1.73743   -0.523541   2.13401    1.59191  

In [15]:
add(x,y) = x+y
x = 1:10
y = 2:11
x .+ y'


10×10 Array{Int64,2}:
  3   4   5   6   7   8   9  10  11  12
  4   5   6   7   8   9  10  11  12  13
  5   6   7   8   9  10  11  12  13  14
  6   7   8   9  10  11  12  13  14  15
  7   8   9  10  11  12  13  14  15  16
  8   9  10  11  12  13  14  15  16  17
  9  10  11  12  13  14  15  16  17  18
 10  11  12  13  14  15  16  17  18  19
 11  12  13  14  15  16  17  18  19  20
 12  13  14  15  16  17  18  19  20  21

### Broadcast loop fusion

In [16]:
f(x,y,z) = sum(exp.(x .- y) .* z .- x) 

f (generic function with 1 method)

In [17]:
x,y,z = rand(1000), rand(1000), rand(1000);

In [19]:
@time f(x,y,z)

  0.000070 seconds (6 allocations: 8.109 KiB)


51.88503188008711

### Exercises

1. Try broadcasting some functions